In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

!python -m spacy download en

     |████████████████████████████████| 44 kB 1.8 MB/s 
     |████████████████████████████████| 13.5 MB 9.8 MB/s 
     |████████████████████████████████| 6.3 MB 25.4 MB/s 
     |████████████████████████████████| 71 kB 12.6 MB/s 
     |████████████████████████████████| 188 kB 99.7 MB/s 
     |████████████████████████████████| 456 kB 72.2 MB/s 
     |████████████████████████████████| 42 kB 2.0 MB/s 
     |████████████████████████████████| 10.1 MB 78.5 MB/s 
     |████████████████████████████████| 623 kB 96.0 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall

In [ ]:
import spacy
import scispacy
import pandas as pd
import numpy as np
import pickle as pk
from pprint import pprint
import os

from spacy import displacy
import en_core_sci_sm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import OrderedDict,Counter

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def _read_medline_abstracts_w_entities(icd_chapter):
  MEDLINE_ABSTRACTS_PATH = "medline_abstracts_w_entities"
  df = pk.load( open(os.path.join(MEDLINE_ABSTRACTS_PATH, f"{icd_chapter}.pk"), "rb"))

  return df

In [ ]:
def entity_linker(linker_name, entity_name):
  nlp = spacy.load("en_ner_bc5cdr_md")
  nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": linker_name})
  doc = nlp(entity_name)
  
  dict_cui_entity = {}
  try:
    entity = doc.ents[0]
  except IndexError:
    entity = 'NaN'
  try:
    linker = nlp.get_pipe("scispacy_linker")
    for umls_ent in entity._.kb_ents:
      cui_id = linker.kb.cui_to_entity[umls_ent[0]].concept_id
      umls_canonical_name = linker.kb.cui_to_entity[umls_ent[0]].canonical_name

      dict_cui_entity[cui_id] = umls_canonical_name
  except AttributeError:
    pass
  
  return dict_cui_entity


In [ ]:
icd_chapter = "certain_infectious_or_parasitic_diseases_Medline"


dict_cluster_label_entities = _read_medline_abstracts_w_entities(icd_chapter)
df = _read_medline_abstracts_w_entities(icd_chapter).drop(columns=['Named_Entities_mimic_w_i2b2'])


In [ ]:
df.head()

,PMID,Title,Abstract,Named_Entities_genia_w_BC5CDR
0,33743560,Early complications of extracorporeal shockwav...,The aim of the study was to analyse the freque...,"{'DISEASE': ['urolithiasis', 'urinary obstruct..."
1,33743553,Seroprevalence of rubella virus among pregnant...,OBJECTIVES: This study aimed to determine rube...,"{'DISEASE': ['rubella virus infectivity', 'rub..."
2,33743550,Responsible Inclusion of Pregnant Individuals ...,Hepatitis C virus (HCV) infections have increa...,{'DISEASE': ['Hepatitis C virus (HCV) infectio...
3,33743545,Brucellosis in Transplant Recipients: A System...,"BACKGROUND: Brucellosis, a bacterial disease c...","{'DISEASE': ['Brucellosis', 'bacterial disease..."
4,33743541,A Prospective Study Evaluating Changes in Hist...,INTRODUCTION: Histological and clinical outcom...,"{'DISEASE': ['coinfection', 'HBV-HIV co-infect..."


In [ ]:
KB_linkers = ['umls', 'hpo', 'rxnorm', 'mesh']
linker_name = KB_linkers[0]

lst_idx = 0
disease_idx = 6

#entity_name = df['Named_Entities_genia_w_BC5CDR'].values[lst_idx]['DISEASE'][disease_idx]
entity_name = "coronary artery disease"

print(entity_name)
dict_cui_entity = entity_linker(linker_name, entity_name)

coronary artery disease
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmp7j1xtk4m
Finished download, copying /tmp/tmp7j1xtk4m to cache at /root/.scispacy/datasets/e9f7327283e43f0482f7c0c71b71dec278a58ccb3ffdd03c2c2350159e7ef146.f2a350ad19015b2591545f7feeed6a6d6d2fffcd635d868a5d7fc0dfc3cadfd8.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmp3ljg_g5z
Finished download, copying /tmp/tmp3ljg_g5z to cache at /root/.scispacy/datasets/f48455d6c79262057cce66b4619123c2b558b21092d42fac97f47bb99a5b8f9f.dd70d3dffe7d90d7ac8914460e16a48375dab32485fb6313a34e6fbcaf53218b.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmp_iw5h6bc
Finished download, copying /tmp/tmp_iw5h6bc to cac

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/concept_aliases.json not found in cache, downloading to /tmp/tmpnve4ggsv
Finished download, copying /tmp/tmpnve4ggsv to cache at /root/.scispacy/datasets/1428ec15d3b1061731ea273c03699130b3d6b90948993e74bda66af605ff8e2a.aeb7a686c654df6bccb6c2c23d3eda3eb381daaefda4592b58158d0bee53b352.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/kbs/2020-10-09/umls_2020_aa_cat0129.jsonl not found in cache, downloading to /tmp/tmppx7z1byg
Finished download, copying /tmp/tmppx7z1byg to cache at /root/.scispacy/datasets/4d7fb8fcae1035d1e0a47d9072b43d5a628057d35497fbfb2499b4b7b2dd4dd7.05ec7eef12f336d4666da85b7fa69b9401883a7dd4244473f7b88b413ccbba03.umls_2020_aa_cat0129.jsonl
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmpztu685za
Finished download, copying /tmp/tmpztu685za to cache at /root/.scispacy/datasets/21a1012c53

/usr/local/lib/python3.7/dist-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/usr/local/lib/python3.7/dist-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [ ]:
dict_cui_entity

{'C0010054': 'Coronary Arteriosclerosis',
 'C0010068': 'Coronary heart disease',
 'C0206064': 'Microvascular Angina',
 'C1956346': 'Coronary Artery Disease',
 'C3640263': 'CDISC SDTM Coronary Artery Disease Risk Terminology'}

In [ ]:
lst_idx = 0
disease_idx = 8

entity_name = df['Named_Entities_genia_w_BC5CDR'].values[lst_idx]['DISEASE'][disease_idx]

print(entity_name)
dict_cui_entity = entity_linker(linker_name, entity_name)

urinary obstruction


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/usr/local/lib/python3.7/dist-packages/scispacy/candida

In [ ]:
dict_cui_entity

{'C0005694': 'Bladder neck obstruction',
 'C0178879': 'Urinary tract obstruction',
 'C4552953': 'Urinary Tract Obstruction, CTCAE',
 'C4685449': 'Grade 1 Urinary Tract Obstruction, CTCAE',
 'C4686000': 'Grade 2 Urinary Tract Obstruction, CTCAE'}

In [ ]:
lst_idx = 3
disease_idx = 0

entity_name = df['Named_Entities_genia_w_BC5CDR'].values[lst_idx]['DISEASE'][disease_idx]

print(entity_name)
dict_cui_entity = entity_linker(linker_name, entity_name)

Brucellosis


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/usr/local/lib/python3.7/dist-packages/scispacy/candida

In [ ]:
dict_cui_entity

{'C0006303': 'Brucella',
 'C0006309': 'Brucellosis',
 'C0302363': 'Brucella abortus infection',
 'C2231324': 'Brucellosis, Pulmonary'}

In [ ]:
entity_name = "heart failure"

print("Named Entity: ", entity_name)

dict_cui_entity = entity_linker(linker_name, entity_name)
print("Corresponding entities in UMLS KB:")
print(dict_cui_entity)

Named Entity:  heart failure
Corresponding entities in UMLS KB:
{'C0018801': 'Heart failure', 'C0018802': 'Congestive heart failure', 'C4554158': 'Heart Failure, CTCAE', 'C0023212': 'Left-Sided Heart Failure', 'C2939447': 'Right ventricular failure'}


# **Distant Supervision using UMLS via OwlReady2**

In [ ]:
!pip3 --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
!pip3 install Owlready2

     |████████████████████████████████| 23.7 MB 1.1 MB/s 
  Created wheel for Owlready2: filename=Owlready2-0.34-cp37-cp37m-linux_x86_64.whl size=20419496 sha256=80ebd5b4f119132331ceeda535a0756b1da1b643ebb6db532351a604e0dbd5ed
  Stored in directory: /root/.cache/pip/wheels/ec/bd/94/4b682aa2b2c7a9d3cd5c7e036f7267940a92ef55889de68db6
Successfully built Owlready2


In [ ]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *

In [ ]:
default_world.set_backend(filename = "pym.sqlite3")

In [ ]:
import_umls("umls-2021AA-full.zip", terminologies = ["ICD10", "SNOMEDCT_US", "CUI"])

Importing UMLS from Zip file umls-2021AA-full.zip with Python version 3.7...
Full UMLS release - importing UMLS from inner Zip file 2021AA-full/2021aa-1-meta.nlm...
  Parsing 2021AA/META/MRRANK.RRF.gz as MRRANK with encoding UTF-8
  Parsing 2021AA/META/MRCONSO.RRF.aa.gz as MRCONSO with encoding UTF-8
  Parsing 2021AA/META/MRCONSO.RRF.ab.gz as MRCONSO with encoding UTF-8
  Parsing 2021AA/META/MRDEF.RRF.gz as MRDEF with encoding UTF-8
  Parsing 2021AA/META/MRSTY.RRF.gz as MRSTY with encoding UTF-8
Full UMLS release - importing UMLS from inner Zip file 2021AA-full/2021aa-2-meta.nlm...
  Parsing 2021AA/META/MRREL.RRF.aa.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ab.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ac.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ad.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRSAT.RRF.aa.gz as MRSAT with encoding UTF-8
  Parsing 2021AA/META/MRSAT.RRF.ab.gz as MRSAT with encoding UTF-8
  Parsing

get_ontology("http://PYM/")

In [ ]:
PYM = get_ontology("http://PYM/").load()

In [ ]:
default_world.save()

In [ ]:
CUI = PYM["CUI"]

### All terminologies across ICD-10, SNOMED-CT, and SNOMED-CT

In [ ]:
PYM.search("depression anxiety")

[SNOMEDCT_US["273402009"] # Depression anxiety scale
, SNOMEDCT_US["273562009"] # Leeds scales for the self-assessment of anxiety and depression
, SNOMEDCT_US["416767005"] # Depression anxiety stress scales stress score
, SNOMEDCT_US["416954004"] # Depression anxiety stress scales depression score
, SNOMEDCT_US["416619008"] # Depression anxiety stress scales anxiety score
, SNOMEDCT_US["445043005"] # Marks and Mathews fear questionnaire total associated anxiety and depression score
, SNOMEDCT_US["445991008"] # Assessment using hospital anxiety and depression scale
, SNOMEDCT_US["273524006"] # Hospital anxiety and depression scale
, SNOMEDCT_US["716763004"] # Assessment using RCADS (Revised Children's Anxiety and Depression Scale)
, SNOMEDCT_US["718747002"] # RCADS (Revised Children's Anxiety and Depression Scale) generalized anxiety subscale
, SNOMEDCT_US["718746006"] # RCADS (Revised Children's Anxiety and Depression Scale) separation anxiety subscale
, SNOMEDCT_US["718745005"] # RCAD

In [ ]:
concept = CUI["C0085580"]

In [ ]:
concept

CUI["C0085580"] # Essential hypertension

### Search just the UMLS Dictionary

In [ ]:
CUI["C5231206"]

CUI["C5231206"] # Hypertension stage 1

### Experiment for the entities in an entity cluster

In [ ]:
import pickle as pk
from pprint import pprint

ENTITY_PATH = "entity_clusters"
entity_cluster = pk.load(open(f"{ENTITY_PATH}/neoplasms_Medline.pk", "rb"))

In [ ]:
#pprint(entity_cluster)

In [ ]:
PYM.search("sickle cell disease pain")

[SNOMEDCT_US["769167005"] # Vaso-occlusive pain episode in sickle cell disease
, CUI["C4708695"] # Vaso-occlusive pain episode in sickle cell disease
]

In [ ]:
PYM.search("coronavirus pneumonia")

[SNOMEDCT_US["441590008"] # Pneumonia due to Severe acute respiratory syndrome coronavirus
, SNOMEDCT_US["713084008"] # Pneumonia caused by Human coronavirus
, CUI["C1260415"] # Pneumonia due to Severe acute respiratory syndrome coronavirus
, CUI["C4075701"] # Pneumonia caused by Human coronavirus
, SNOMEDCT_US["882784691000119100"] # Pneumonia caused by SARS-CoV-2
, CUI["C5244027"] # Pneumonia caused by SARS-CoV-2
]

In [ ]:
PYM.search("tumor melanoma")

[SNOMEDCT_US["399397007"] # pT2: Tumor greater than 10 mm but not more than 16 mm in greatest basal diameter and between 2.5 and 10 mm in maximum height (thickness) (ciliary body and choroid) (uveal melanoma)
, SNOMEDCT_US["399448002"] # pT1: Tumor 10 mm or less in greatest diameter and 2.5 mm or less in greatest height (thickness) (ciliary body and choroid) (uveal melanoma)
, SNOMEDCT_US["399461002"] # pT1b: Tumor 10 mm or less in greatest diameter and 2.5 mm or less in greatest height (thickness) with microscopic extraocular extension (ciliary body and choroid) (uveal melanoma)
, SNOMEDCT_US["399474008"] # pT2a: Tumor confluent with or extending into the ciliary body and/or choroid with melanomalytic glaucoma (iris) (uveal melanoma)
, SNOMEDCT_US["399493005"] # pT1c: Tumor 10 mm or less in greatest diameter and 2.5 mm or less in greatest height (thickness) with macroscopic extraocular extension (ciliary body and choroid) (uveal melanoma)
, SNOMEDCT_US["399501001"] # pT1: Tumor limite

## Download wikidata json dump. For documentation, look at 
https://dumps.wikimedia.org/wikidatawiki/entities/latest-all.json.bz2

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
!wget https://dumps.wikimedia.org/wikidatawiki/entities/latest-all.json.bz2

--2021-09-20 23:23:53--  https://dumps.wikimedia.org/wikidatawiki/entities/latest-all.json.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70126370250 (65G) [application/octet-stream]
Saving to: ‘latest-all.json.bz2’

latest-all.json.bz2  74%[=============>      ]  48.75G  4.96MB/s    in 2h 51m  

2021-09-21 02:15:49 (4.84 MB/s) - Connection closed at byte 52349697706. Retrying.

--2021-09-21 02:15:50--  (try: 2)  https://dumps.wikimedia.org/wikidatawiki/entities/latest-all.json.bz2
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 70126370250 (65G), 17776672544 (17G) remaining [application/octet-stream]
Saving to: ‘latest-all.json.bz2’

latest-all.json.bz2  92%[++++++++++++++===>  ]  60.34G